In [1]:
#%load_ext autoreload
#%autoreload 2

#import sys
#sys.path.insert(0,'..')



import os

import logging


import time
import psutil
from subprocess import PIPE
import multiprocessing

import tensorflow as tf

from btgym import BTgymEnv, BTgymDataset

from launcher import Launcher

In [2]:
cluster_config = dict(
    host='127.0.0.1',
    port=12222,
    num_workers=2,
    num_ps=1,
    log_dir='./a3c_log',
)

# Provide data:
MyDataset = BTgymDataset(
    filename='../data/DAT_ASCII_EURUSD_M1_2016.csv',
    start_weekdays=[0, 1, 2, 3, 4],
    episode_len_days=0,
    episode_len_hours=23,
    episode_len_minutes=0,
    start_00=False,
    time_gap_hours=2,
)

env_config = dict(
    dataset=MyDataset,
    port=5000,
    data_port=4999,
    connect_timeout=5,
    verbose=1,
)

launcher = Launcher(
    cluster_config=cluster_config,
    env_class=BTgymEnv,
    env_config=env_config,
    
)

[2017-08-11 13:12:31,180] ./a3c_log created.
[2017-08-11 13:12:31,468] ready.


In [ ]:
launcher.env_config

In [3]:
launcher.run()

[2017-08-11 13:12:44,710] tf.server started.
[2017-08-11 13:12:44,710] tf.server started.
[2017-08-11 13:12:44,710] parameters_server started.
[2017-08-11 13:12:44,726] making environment.
[2017-08-11 13:12:44,727] making environment.
[2017-08-11 13:12:44,781] Connecting data_server...
[2017-08-11 13:12:44,781] Custom Dataset class used.
[2017-08-11 13:12:44,783] Connecting data_server...
[2017-08-11 13:12:44,891] DataServer PID: 13664
[2017-08-11 13:12:45,733] Loaded 372678 records from <../data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2017-08-11 13:12:45,869] Data summary:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092

INFO:tensorflow:Starting standard services.


[2017-08-11 13:12:47,319] Starting standard services.


INFO:tensorflow:Starting queue runners.


[2017-08-11 13:12:47,333] Starting queue runners.


Trainer_0: sync`ed.
Trainer_0 started.


[2017-08-11 13:12:47,337] Starting training at step=0


Traner_0: processed step 0
Traner_0: processed step 1
Traner_0: processed step 2
Traner_0: processed step 3
Traner_0: processed step 4
Traner_0: processed step 5
Traner_0: processed step 6
Traner_0: processed step 7
Traner_0: processed step 8
Traner_0: processed step 9


[2017-08-11 13:12:47,355] Server is exiting.
[2017-08-11 13:12:47,357] Server is exiting. Exit code: None
[2017-08-11 13:12:47,359] DataServer is exiting.
[2017-08-11 13:12:47,361] DataServer is exiting. Exit code: None
[2017-08-11 13:12:47,363] Environment closed.
[2017-08-11 13:12:47,365] reached 10 steps, exiting.
[2017-08-11 13:12:47,376] worker_0 has joined.
[2017-08-11 13:12:47,378] worker_1 has joined.
[2017-08-11 13:12:47,383] parameter_server_0 has joined.
[2017-08-11 13:12:47,385] launcher closed.
